# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [1]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

  Cloning git://github.com/Theano/Theano.git to /tmp/pip-req-build-yysuif09
  Running command git clone -q git://github.com/Theano/Theano.git /tmp/pip-req-build-yysuif09
  Created wheel for Theano: filename=Theano-1.0.4+21.g8f510a1f0-cp37-none-any.whl size=2667497 sha256=908ec45bd442ff17524f71583186eb96580eda9fd744ee5ddd66c24b58971f91
  Stored in directory: /tmp/pip-ephem-wheel-cache-fh1qqj5y/wheels/ae/32/7c/62beb8371953eb20c271b3bac7d0e56e1a2020d46994346b52
Successfully built Theano
  Found existing installation: Theano 1.0.4+21.g8f510a1f0
    Uninstalling Theano-1.0.4+21.g8f510a1f0:
      Successfully uninstalled Theano-1.0.4+21.g8f510a1f0
Requirement already up-to-date: tensorflow in /srv/conda/envs/notebook/lib/python3.7/site-packages (2.0.0)
Requirement already up-to-date: keras in /srv/conda/envs/notebook/lib/python3.7/site-packages (2.3.1)
Requirement already up-to-date: pip in /srv/conda/envs/notebook/lib/python3.7/site-packages (19.3.1)
Requirement already up-to-date: pandas i

# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [8]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softmax', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [9]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 40)

Epoch 1/40
8000/8000 [==============================] - 2s 278us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 2/40
8000/8000 [==============================] - 2s 274us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 3/40
8000/8000 [==============================] - 2s 288us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 4/40
8000/8000 [==============================] - 2s 272us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 5/40
8000/8000 [==============================] - 2s 265us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 6/40
8000/8000 [==============================] - 2s 259us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 7/40
8000/8000 [==============================] - 2s 228us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 8/40
8000/8000 [==============================] - 2s 248us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 9/40
8000/8000 [==============================] - 2s 251us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 10/40
8000/8000 [==============================] - 2s 298u

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [10]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [12]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 235us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 2s 234us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 210us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 218us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 2s 200us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 2s 202us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 2s 195us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 2s 190us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 159us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================]

In [13]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


# Initialising the 3rd ANN

In [15]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 205us/step - loss: 0.5057 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 173us/step - loss: 0.4448 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 208us/step - loss: 0.4328 - accuracy: 0.7972
Epoch 4/100
8000/8000 [==============================] - 2s 200us/step - loss: 0.4288 - accuracy: 0.8015
Epoch 5/100
8000/8000 [==============================] - 2s 215us/step - loss: 0.4280 - accuracy: 0.8002
Epoch 6/100
8000/8000 [==============================] - 2s 206us/step - loss: 0.4277 - accuracy: 0.8048
Epoch 7/100
8000/8000 [==============================] - 2s 207us/step - loss: 0.4274 - accuracy: 0.8029
Epoch 8/100
8000/8000 [==============================] - 2s 222us/step - loss: 0.4275 - accuracy: 0.8046
Epoch 9/100
8000/8000 [==============================] - 2s 225us/step - loss: 0.4273 - accuracy: 0.8039
Epoch 10/100
8000/8000 [==============================]

In [16]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1519   76]
 [ 239  166]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [20]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print("Acurácia:")
print(accuracies)
print("Média:")
print(mean)
print("Variância:")
print(variance)

Epoch 1/10
7200/7200 [==============================] - 2s 212us/step - loss: 0.5037 - accuracy: 0.7962
Epoch 2/10
7200/7200 [==============================] - 2s 230us/step - loss: 0.4308 - accuracy: 0.7971
Epoch 3/10
7200/7200 [==============================] - 2s 244us/step - loss: 0.4276 - accuracy: 0.7971
Epoch 4/10
7200/7200 [==============================] - 2s 222us/step - loss: 0.4237 - accuracy: 0.7971
Epoch 5/10
7200/7200 [==============================] - 2s 242us/step - loss: 0.4205 - accuracy: 0.8046
Epoch 6/10
7200/7200 [==============================] - 2s 248us/step - loss: 0.4182 - accuracy: 0.8228
Epoch 7/10
7200/7200 [==============================] - 2s 261us/step - loss: 0.4163 - accuracy: 0.8256
Epoch 8/10
7200/7200 [==============================] - 2s 220us/step - loss: 0.4147 - accuracy: 0.8282
Epoch 9/10
7200/7200 [==============================] - 2s 261us/step - loss: 0.4130 - accuracy: 0.8301
Epoch 10/10
800/800 [==============================] - 0s 100us/

# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [21]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

Epoch 1/10
7200/7200 [==============================] - 2s 281us/step - loss: 0.4943 - accuracy: 0.7969
Epoch 2/10
7200/7200 [==============================] - 2s 257us/step - loss: 0.4345 - accuracy: 0.7971
Epoch 3/10
7200/7200 [==============================] - 2s 265us/step - loss: 0.4320 - accuracy: 0.7971
Epoch 4/10
7200/7200 [==============================] - 2s 247us/step - loss: 0.4296 - accuracy: 0.7971
Epoch 5/10
7200/7200 [==============================] - 2s 278us/step - loss: 0.4300 - accuracy: 0.7971
Epoch 6/10
7200/7200 [==============================] - 2s 246us/step - loss: 0.4263 - accuracy: 0.7971
Epoch 7/10
7200/7200 [==============================] - 2s 242us/step - loss: 0.4273 - accuracy: 0.8062
Epoch 8/10
7200/7200 [==============================] - 2s 232us/step - loss: 0.4263 - accuracy: 0.8188
Epoch 9/10
7200/7200 [==============================] - 2s 238us/step - loss: 0.4284 - accuracy: 0.8222
Epoch 10/10
800/800 [==============================] - 0s 170us/

# Tuning the ANN

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier('rmsprop'))

In [ ]:
#parameters = {'batch_size': [10, 24, 32],
#              'epochs': [50, 100, 200],
#              'optimizer': ['adam', 'rmsprop']}

#grid_search = GridSearchCV(estimator = classifier,
#                           scoring = 'accuracy',
#                           param_grid = parameters,
#                           cv = 10)

#grid_search = grid_search.fit(X_train, y_train)

#best_parameters = grid_search.best_params_
#best_accuracy = grid_search.best_score_

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)